In [ ]:
import numpy as np
import pandas as pd
import keras
import cv2
from matplotlib import pyplot as plt
import os
import random
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from keras.applications import VGG19
from keras.layers import *
from keras.models import Sequential

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train.csv")
base_path = "./images/"

In [ ]:
samples = 5000000
df = df.loc[df["id"].str.startswith('00', na=False)]
num_classes = len(df["landmark_id"].unique())
num_data = len(df

In [ ]:
data = pd.DataFrame(df["landmark_id"].value_counts())
data.reset_index(inplace=True)
data.head()

In [ ]:
data.tail()

In [ ]:
data.columns=["landmark_id","count"]

In [ ]:
data["landmark_id"].describe()

In [ ]:
plt.hist(data["count"], 100, range=(0,58),label="test")

data["count"].between(0,5).sum()

data["count"].between(5,10).sum()

plt.hist(df["landmark_id"], bins=len(df["landmark_id"].unique()))

lencoder = LabelEncoder()
lencoder.fit(df["landmark_id"])

In [ ]:
df.head()

In [ ]:
def encode_label(lbl):
    return lencoder.transform(lbl)

In [ ]:
def decode_label(lbl):
    return lencoder.inverse_transform(lbl)

In [ ]:
fig = plt.figure(figsize=(16, 16))
for i in range(1,5):
    print(os.listdir(base_path))
    ri = random.choices(os.listdir(base_path), k=3)
    if len(ri) < 3:
        continue  # Skip this iteration if there are not enough elements
    folder = os.path.join(base_path, ri[0], ri[1], ri[2])
    if not os.path.isdir(folder):
        continue  # Skip this iteration if the folder does not exist
    random_img = random.choice(os.listdir(folder))
    img = np.array(Image.open(os.path.join(folder, random_img)))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    fig.add_subplot(1, 4, i)
    plt.imshow(img)
    plt.axis('off')

In [ ]:
plt.show()

In [ ]:
source_model = VGG19(weights=None)
drop_layer = Dropout(0.5)
drop_layer2 = Dropout(0.5)

In [ ]:
model = Sequential()
for layer in source_model.layers[:-1]:
    if layer == source_model.layers[-25]:
        model.add(BatchNormalization())
model.add(Dense(num_classes, activation="softmax"))
model.summary()

In [ ]:
learning_rate = 0.0001
decay_speed = 1e-6
momentum = 0.09
loss_function = "sparse_categorical_crossentropy"
optim1 = keras.optimizers.RMSprop(lr=learning_rate)
model.compile(optimizer=optim1,
              loss=loss_function,
              metrics=["accuracy"])

In [ ]:
def image_reshape(im, target_size):
    return cv2.resize(im, target_size)

In [ ]:
def get_batch(dataframe, start, batch_size):
    image_array = []
    label_array = []

In [ ]:
 end_img = start + batch_size
    if end_img > len(dataframe):
        end_img = len(dataframe)

In [ ]:
  for idx in range(start, end_img):
        n = idx
        im, label = get_image_from_number(n, dataframe)
        im = image_reshape(im, (224, 224)) / 255.0
        image_array.append(im)
        label_array.append(label)
        label_array = encode_label(label_array)
        return np.array(image_array), np.array(label_array)

In [ ]:
batch_size = 16
epoch_shuffle = True
weight_classes = True
epochs = 1

In [ ]:
train, val = np.split(df.sample(frac=1), [int(0.8 * len(df))])
print(len(train))
print(len(val))

In [ ]:
for e in range(epochs):
    print("Epoch : " + str(e + 1) + "/" + str(epochs))
    if epoch_shuffle:
        train = train.sample(frac=1)  # Corrected 'shample' to 'sample'
    for it in range(int(np.ceil(len(train) / batch_size))):
        x_train, y_train = get_batch(train, it * batch_size, batch_size)
        model.train_on_batch(x_train, y_train)

In [ ]:
model.save("Model.h5")
batch_size = 16
errors = 0
good_preds = []
bad_preds = []

In [ ]:
for it in range(int(np.ceil(len(val) / batch_size))):
    x_val, y_val = get_batch(val, it * batch_size, batch_size)
    result = model.predict(x_val)
    cla = np.argmax(result, axis=1)  # Corrected 'axis-1' to 'axis=1'
    for idx, res in enumerate(result):
        if cla[idx] != y_val[idx]:
            errors = errors + 1
            bad_preds.append([batch_size * it + idx, cla[idx], res[cla[idx]]])
        else:
            good_preds.append([batch_size * it + idx, cla[idx], res[cla[idx]]])

In [ ]:
for i in range(1, 16):
    n = int(good_preds[0])
    img, lbl = get_image_from_number(n, val)
    img = cv2